In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader,TensorDataset,random_split
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [93]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 사용 디바이스: {device}")

🔥 사용 디바이스: cuda


### 데이터 로드

In [94]:
housing = fetch_california_housing()

In [95]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [96]:
x = housing.data
y = housing.target
feature_name = housing.feature_names

In [97]:
x.shape

(20640, 8)

In [98]:
y.shape

(20640,)

In [99]:
feature_name

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

# 선형회귀 모델

## 모델 선언

In [100]:
class LinearRegressionModel(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.linear=nn.Linear(input_size, 1)
    def forward(self,x):
        return self.linear(x)

In [101]:
uni_model = LinearRegressionModel(1).to(device)

In [102]:
mul_model = LinearRegressionModel(x.shape[1]).to(device)

## Feature Scaling

단변량용

In [103]:
room_idx = feature_name.index('AveRooms')

In [104]:
x_uni = x[:,room_idx].reshape(-1,1)

In [105]:
x_uni

array([[6.98412698],
       [6.23813708],
       [8.28813559],
       ...,
       [5.20554273],
       [5.32951289],
       [5.25471698]])

다변량용

In [106]:
x_mul = x.copy()

In [107]:
x_mul

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

StandardScaler - 사이킷런에서 제공하는 데이터 전처리 도구로,
특성(피처)들의 값을 평균 0, 표준편차 1이 되도록 표준화(정규화)해줍니다.

스케일링은 다른 변수와 보폭을 맞추는 느낌이므로 단변량은 적용을 안하도록 하겠습니다.

In [108]:
scaler = StandardScaler()

In [109]:
x_mul_scaled = scaler.fit_transform(x_mul)

In [110]:
x_mul_scaled

array([[ 2.34476576,  0.98214266,  0.62855945, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 2.33223796, -0.60701891,  0.32704136, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.7826994 ,  1.85618152,  1.15562047, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [-1.14259331, -0.92485123, -0.09031802, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [-1.05458292, -0.84539315, -0.04021111, ..., -0.09122515,
         1.77823747, -0.87362627],
       [-0.78012947, -1.00430931, -0.07044252, ..., -0.04368215,
         1.75014627, -0.83369581]])

## Dataset 및 DataLoader 구성


### 텐서 변환

In [111]:
x_uni_tensor = torch.FloatTensor(x_uni).to(device)
x_mul_tensor = torch.FloatTensor(x_mul_scaled).to(device)
y_tensor = torch.FloatTensor(y.reshape(-1,1)).to(device)

한 행에 값이 나열된 1차원 벡터를 한 행에 1개의 값이 들어있도록 변경 그리고 텐서로 변경

In [112]:
y.reshape(-1,1)

array([[4.526],
       [3.585],
       [3.521],
       ...,
       [0.923],
       [0.847],
       [0.894]])

In [113]:
y_tensor

tensor([[4.5260],
        [3.5850],
        [3.5210],
        ...,
        [0.9230],
        [0.8470],
        [0.8940]], device='cuda:0')

### dataset생성

In [114]:
uni_dataset = TensorDataset(x_uni_tensor,y_tensor)
mul_dataset = TensorDataset(x_mul_tensor,y_tensor)

### train/validation 분할

In [115]:
train_size = int(0.8*len(y))
val_size = len(y)-train_size

In [116]:
train_size,val_size

(16512, 4128)

In [117]:
uni_train, uni_val = random_split(uni_dataset,[train_size,val_size])
mul_train, mul_val = random_split(mul_dataset,[train_size,val_size])

### DataLoader생성

In [118]:
batch_size = 256
uni_train_loader = DataLoader(uni_train,batch_size = batch_size,shuffle=True)
uni_val_loader = DataLoader(uni_val,batch_size=batch_size)
mul_train_loader = DataLoader(mul_train,batch_size=batch_size,shuffle=True)
mul_val_loader = DataLoader(mul_val,batch_size=batch_size)

## 학습 함수 정의

In [119]:
def train_model(model,train_loader,val_loader,learning_rate ,epochs ,model_name=""):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(),lr=learning_rate)
    
    train_losses=[]
    val_losses=[]
    
    print(f"{model_name} 학습 시작")
    for epoch in range(epochs):
        model.train()
        epoch_train_loss=0
        
        for batch_x,batch_y in train_loader:
            predictions = model(batch_x)
            loss = criterion(predictions,batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        
        model.eval()
        epoch_val_loss = 0
        
        with torch.no_grad():
            for batch_x,batch_y in val_loader:
                predictions = model(batch_x)
                loss = criterion(predictions,batch_y)
                epoch_val_loss += loss.item()
                
        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_val_loss = epoch_val_loss / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        if (epoch + 1) % 20 == 0:
            print(f"   Epoch {epoch+1:4d}: J(θ) = {avg_train_loss:.4f}, Val J(θ) = {avg_val_loss:.4f}")
        
    return train_losses, val_losses

        

## 모델 학습

In [120]:
uni_train_losses, uni_val_losses = train_model(
    uni_model,uni_train_loader,uni_val_loader,
    learning_rate=0.008, epochs=100, model_name="단변량 모델"
    )

단변량 모델 학습 시작
   Epoch   20: J(θ) = 1.3148, Val J(θ) = 1.2677
   Epoch   40: J(θ) = 1.3200, Val J(θ) = 1.2606
   Epoch   60: J(θ) = 1.3197, Val J(θ) = 1.2707
   Epoch   80: J(θ) = 1.3191, Val J(θ) = 1.2610
   Epoch  100: J(θ) = 1.3186, Val J(θ) = 1.2597


In [121]:
mul_train_losses, mul_val_losses =train_model(
    mul_model, mul_train_loader, mul_val_loader,
    learning_rate=0.008, epochs=100, model_name="다변량 모델"
)

다변량 모델 학습 시작
   Epoch   20: J(θ) = 0.5253, Val J(θ) = 0.5962
   Epoch   40: J(θ) = 0.5162, Val J(θ) = 0.5903
   Epoch   60: J(θ) = 0.5149, Val J(θ) = 0.5887
   Epoch   80: J(θ) = 0.5145, Val J(θ) = 0.5875
   Epoch  100: J(θ) = 0.5152, Val J(θ) = 0.5866


->더 많은 데이터가 더 좋은 예측을 만든다

단일 특성의 한계 = 방 개수만으론 부족
다차원 정보 = 훨씬 정확한 예측